In [ ]:
import pandas as pd
test_data = pd.read_csv("/content/public_test.csv")
test_data

,id,question,option_1,option_2,option_3,option_4,option_5,option_6
0,level3_1,Hương đang mang thai và lo lắng mình có thể gặ...,A. Tuần 10,B.Tuần 20,C. Tuần 30,D. Tuần 40,NaN,NaN
1,level3_2,Hương đang mang thai tuần thứ 5 và lo lắng mìn...,A. 5 tuần,B. 15 tuần,C. 25 tuần,D. 35 tuần,NaN,NaN
2,level3_5,Có bao nhiêu loại rau tiền đạo biết rằng trong...,A. 2,B.3,C. 4,D. 5,NaN,NaN
3,level3_13,Bệnh nhân Dũng được chuẩn đoán bị viêm gan kéo...,Có,Không,NaN,NaN,NaN,NaN
4,level3_14,Một bệnh nhân bị đau tinh hoàn. Sau khi được h...,A. Nhiễm trùng đường tiết niệu,B.Ung thư tinh hoàn,C. Chấn thương,D. Giãn tĩnh mạch thừng tinh,NaN,NaN
...,...,...,...,...,...,...,...,...
95,level4_4,Tiểu Đường là nữ ca sĩ nổi tiếng ở Trung Quốc....,Viêm họng do viêm amidan,Tiểu đường,Ung thư vòm họng,Trĩ nội,NaN,NaN
96,level4_9,Mai hiện mang thai tháng thứ 9. Có rất nhiều b...,Herpes sinh dục,Lậu,Chlamydia,Giang mai,NaN,NaN
97,level4_27,Ông Biền năm nay 73 tuổi. Trong một bữa cơm gi...,U não,Thiếu mãu não,Tắc mạch máu não,Tai biến mạch máu não,NaN,NaN
98,level4_28,U não là tình trạng các khối u hình thành tron...,Đều là các bệnh nguy hiểm,Đều là ung thư,Nguyên nhân chính xác không thể xác định,Xảy ra nhiều nhất ở người già,NaN,NaN


#Simplify the question

In [ ]:
!pip install --quiet deep_translator
!pip install --quiet ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.4 MB/s eta 0:00:00
time: 371 µs (started: 2023-10-27 15:34:19 +00:00)


In [ ]:
from deep_translator import GoogleTranslator
import spacy

time: 13.3 s (started: 2023-10-27 15:34:19 +00:00)


In [ ]:
def sentence_parsing(text):
  vi_eng_translator = GoogleTranslator(source='vi', target='en')
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(text)
  new_text = [text]
  for chunk in doc.noun_chunks:
    if chunk.root.dep_ == 'appos' and chunk.root.head.dep_ == 'dobj':
      new_text = text.split(chunk.text)
  return [vi_eng_translator.translate(text) for text in new_text]

sentence_parsing(test_data['question'][2])

['How many types of vegetable strikers are there?',
 'In soccer, each team usually has a maximum of 3 strikers on the field?']

time: 2.23 s (started: 2023-10-27 15:34:32 +00:00)


In [ ]:
def clause_parsing(text):
  nlp = spacy.load('en_core_web_sm')
  vi_eng_translator = GoogleTranslator(source='vi', target='en')
  eng_text = vi_eng_translator.translate(text)
  doc = nlp(eng_text)
  new_text = [eng_text]
  for token in doc:
    if token.pos_ == 'SCONJ' and token.dep_ == 'mark':
      new_text = eng_text.split(token.text)
  return new_text

clause_parsing(test_data['question'][61])

['What effects do antithyroid drugs have?']

time: 1.06 s (started: 2023-10-27 15:34:35 +00:00)


In [ ]:
def single_quest_split(text):
  text_lst = sentence_parsing(text)
  if len(text_lst) == 1:
    text_lst = clause_parsing(text)
  return text_lst

single_quest_split(test_data['question'][2])

['How many types of vegetable strikers are there?',
 'In soccer, each team usually has a maximum of 3 strikers on the field?']

time: 1.2 s (started: 2023-10-27 15:34:36 +00:00)


In [ ]:
def split_quest(text):
  text_list = text.split('. ')
  if len(text_list) == 1:
    new_text_list = single_quest_split(text_list[0])
    return [i for i in new_text_list if i != '']
  else:
    new_text_list = []
    for txt in text_list:
      new_text_list += single_quest_split(txt)
    return [i for i in new_text_list if i != '']

split_quest(test_data['question'][9])

['A healthy person who eats chicken combs will not experience any unusual symptoms in the abdomen, thighs, or inside the anus.',
 'Does that person have genital warts?']

time: 8.19 s (started: 2023-10-27 15:34:37 +00:00)


In [ ]:
def ques_processing(text):
  text_list = split_quest(text)[::-1]

  #vi_eng_translator = GoogleTranslator(source='vi', target='en')
  #eng_text_list = [vi_eng_translator.translate(text) for text in text_list]

  stopwords = spacy.lang.en.stop_words.STOP_WORDS
  wh_words_list = ['what', 'whatever', 'who', 'whoever', 'whom', 'whomever', 'whose', 'which',
                  'whichever', 'when', 'whenever', 'where', 'wherever', 'why', 'how', 'however']
  new_stopwords = [word for word in stopwords if word not in wh_words_list]

  preprocessed_text = []
  if len(text_list) > 2:
    for eng_text in text_list:
      nlp = spacy.load('en_core_web_sm')
      remove_propn = ' '.join([token.text for token in nlp(eng_text) if token.pos_ not in ["PROPN"]])
      preprocessed_text.append(remove_propn)
      #lemmas = [token.lemma_ for token in nlp(remove_propn)]
      #preprocessed_text.append(' '.join([lemma for lemma in lemmas if lemma not in new_stopwords]))
  else:
      preprocessed_text.append(' '.join(text_list))

  #eng_vi_translator = GoogleTranslator(source='en', target='vi')
  #new_text = '. '.join([eng_vi_translator.translate(text) for text in preprocessed_text])

  return '. '.join(preprocessed_text).lower()

ques_processing(test_data['question'][10])

'mr. than wants to check images of his heart and lungs. which of the following methods can he use?'

time: 4.01 s (started: 2023-10-27 15:34:45 +00:00)


#Get keyword

In [ ]:
!gdown 1-Tpg2Bp5Yl9henrlTlinEFwUp0hFhblv

Downloading...
From: https://drive.google.com/uc?id=1-Tpg2Bp5Yl9henrlTlinEFwUp0hFhblv
To: /content/final_answer_dict.csv
100% 8.29M/8.29M [00:00<00:00, 136MB/s]
time: 2.73 s (started: 2023-10-27 15:34:49 +00:00)


In [ ]:
import pandas as pd
corpus = pd.read_csv("/content/final_answer_dict.csv")

time: 208 ms (started: 2023-10-27 15:34:52 +00:00)


In [ ]:
!gdown 1dGkMLzXbg-n4q5x8rLCcwy9z9azlHA5O

Downloading...
From: https://drive.google.com/uc?id=1dGkMLzXbg-n4q5x8rLCcwy9z9azlHA5O
To: /content/final_corpus_keys.txt
100% 116k/116k [00:00<00:00, 21.1MB/s]
time: 1.11 s (started: 2023-10-27 17:16:04 +00:00)


In [ ]:
corpus_keys = pd.read_csv("/content/final_corpus_keys.txt", sep=",", header=None).iloc[0].tolist()
corpus_keys[:7]

['ultrasound image',
 'milk duct blockage',
 'diagnose hypospadias',
 'fournier disease contagious',
 'small bumps',
 'notes',
 'prodrome stage']

time: 352 ms (started: 2023-10-27 17:16:11 +00:00)


In [ ]:
!pip install -q git+https://github.com/boudinfl/pke.git

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.6 MB/s eta 0:00:00
time: 14 s (started: 2023-10-27 15:34:52 +00:00)


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import traceback
import pke
import re
import ast

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


time: 10.5 s (started: 2023-10-27 15:35:06 +00:00)


In [ ]:
!pip -q install Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 977.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.8 MB/s eta 0:00:00
time: 19.4 s (started: 2023-10-27 15:35:17 +00:00)


In [ ]:
corpus_keys = []
for i in range(len(corpus['keyword'])):
  corpus_keys+= ast.literal_eval(corpus['keyword'][i])
corpus_keys_list = list(set(corpus_keys))

time: 335 ms (started: 2023-10-27 17:24:52 +00:00)


In [ ]:
import Levenshtein

time: 30.9 ms (started: 2023-10-27 15:35:55 +00:00)


In [ ]:
def get_nouns_multipartite(text):
  #content = ques_processing(text)
  content = '. '.join(split_quest(text)) #[::-1]
  numbers = re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', content)
  wh_words_list = ['whatever', 'who', 'whoever', 'whom', 'whomever', 'whose', 'which', 'i', 'ii', 'iii', 'iv',
                  'whichever', 'when', 'whenever', 'where', 'wherever', 'however', 'why', 'how', 'can']
  wh_words = [c for c in content.split() if c in wh_words_list]

  out = []
  main_key = []
  key_lst = []
  try:
    #extractor = pke.unsupervised.MultipartiteRank()
    #extractor.load_document(input=content,language='en')
    #not contain punctuation marks or stopwords as candidates.
    #pos = {'PROPN', 'NOUN', 'ADJ'}
    #extractor.candidate_selection(pos=pos)
    #build the Multipartite graph and rank candidates using random walk,
    #alpha controls the weight adjustment mechanism, see TopicRank for threshold/method parameters
    #extractor.candidate_weighting(alpha=1.1, threshold=0.75, method='average')

    # define the set of valid Part-of-Speeches
    pos = {'ADJ', 'PROPN', 'NUM', 'NOUN'}
    # 1. create a SingleRank extractor.
    extractor = pke.unsupervised.SingleRank()
    # 2. load the content of the document.
    extractor.load_document(input=content, language='en', normalization=None)
    # 3. select the longest sequences of nouns and adjectives as candidates.
    extractor.candidate_selection(pos=pos)
    # 4. weight the candidates using the sum of their word's scores that are computed using random walk. In the graph, nodes are words of
    # certain part-of-speech (nouns and adjectives) that are connected if they occur in a window of 10 words.
    extractor.candidate_weighting(window=5, pos=pos)

    keyphrases = extractor.get_n_best(n=20)
    for val in keyphrases:
      #if any([key for key in corpus_keys if Levenshtein.ratio(val[0], key) >= 0.9]):
      if any([key for key in corpus_keys_list if Levenshtein.ratio(val[0], key) >= 0.95]):
        main_key.append(val[0])
        #main_key += val[0].split()
      #if any([key for key in corpus_keys if Levenshtein.ratio(val[0], key) >= 0.7]):
      if any([key for key in corpus_keys_list if Levenshtein.ratio(val[0], key) >= 0.7]):
        out.append(val[0])
        #out += val[0].split()
  except:
    out = []
    traceback.print_exc()

  key_lst = main_key + [o for o in out if o not in main_key]
  #key_lst = main_key + out
  #if len(numbers) > 0:
  #  out += numbers
  #if len(wh_words) > 0:
  #  key_lst += wh_words

  key_lst = [key_lst[0].split() if len(key_lst)==1 else key_lst][0]
  #key_lst = list(set(key_lst))

  return key_lst

time: 1.88 ms (started: 2023-10-27 17:26:44 +00:00)


In [ ]:
for i in range(5,15,1):
  text = test_data['question'][i]
  print(text)
  print(get_nouns_multipartite(text))

An bị viêm cột sống, còn Nam bị tiêu chảy. Hỏi bệnh của Nam thuộc nhóm bệnh lý nào?
['group', 'diseases', 'diarrhea', 'spondylitis']
Việc sử dụng thuốc kháng sinh một cách tự tiện, quá liều lượng, không theo đơn có thể gây ra vấn đề nào?
['non-prescription', 'problems', 'antibiotics', 'arbitrary use']
Đàn ông có thể bị mắc ung thư vú không?
['breast cancer', 'men']
Ung thư buồng trứng là khối u ác tính có xuất phát từ một hoặc cả hai buồng trứng. Theo thống kê tỷ lệ mắc bệnh ung thư buồng trứng khoảng 4,6/100.000 phụ nữ. Bác sĩ dùng cách nào để chẩn đoán căn bệnh này?
['ovarian cancer', 'malignant tumor', 'incidence', 'doctors', 'ovaries', 'statistics', 'disease', 'one']
Một người khỏe mạnh ăn mào gà, không bị xuất hiện các triệu chứng bất thường ở vùng bụng, đùi, bên trong hậu môn. Người đó có phải đã mắc bệnh sùi mào gà không?
['healthy person', 'person', 'unusual symptoms', 'genital warts', 'abdomen', 'thighs', 'anus', 'chicken combs']
Ông Thận muốn kiểm tra hình ảnh tim và phổi của

#Corpus

In [ ]:
!gdown 10l0Utp0tWOLB-ObtO-zFrBFeKhc1I8Gc

Downloading...
From: https://drive.google.com/uc?id=10l0Utp0tWOLB-ObtO-zFrBFeKhc1I8Gc
To: /content/new_corpus.csv
100% 12.2M/12.2M [00:00<00:00, 144MB/s]
time: 3.82 s (started: 2023-10-27 16:03:11 +00:00)


In [ ]:
import pandas as pd
corpus = pd.read_csv("new_corpus.csv")[['context',	'summary',	'keyword']]
corpus

,context,summary,keyword
0,nhiễm ký sinh trùng - nhiễm ký sinh trùng là g...,Parasitic infection is a common disease common...,"['parasitic infection', 'humans', 'subtropical..."
1,nhiễm ký sinh trùng - con đường lây truyền của...,Parasites can be transmitted through digestive...,"['contact', 'animals', 'parasites', 'skin surf..."
2,nhiễm ký sinh trùng - dấu hiệu nhiễm ký sinh t...,"Parasitic infections can be silent, or have wa...","['silent', 'warning signs', 'people', 'parasit..."
3,nhiễm ký sinh trùng - sốt kéo dài. nhiễm ký si...,Parasitic infections often have signs of prolo...,"['fever', 'body chills', 'diarrhea', 'abdomina..."
4,nhiễm ký sinh trùng - rối loạn tiêu hóa. rối l...,"Gastrointestinal parasites can cause diarrhea,...","['diarrhea', 'people', 'flatulence', 'constipa..."
...,...,...,...
9349,nám nội tiết - phương pháp điều trị nám mảng n...,There are many methods to treat endocrine mela...,"['endocrine melasma', 'patients', 'skin', 'tre..."
9350,nám nội tiết - làm thế nào để phòng ngừa bị ná...,"After age 30, women's hormones will gradually ...","['endocrine melasma', 'process', 'stress', 'me..."
9351,bệnh giun lươn - bệnh giun lươn là gì?. bệnh g...,Strongyloides stercoralis was first discovered...,"['strongyloides stercoralis', 'disease', 'late..."
9352,bệnh giun lươn - lâm sàng. hầu hết nhiễm giun ...,Strongyloides infections are often asymptomati...,"['infected soil', 'location', 'larvae', 'aller..."


time: 240 ms (started: 2023-10-27 16:03:15 +00:00)


In [ ]:
!pip install -q rank_bm25
from rank_bm25 import BM25Okapi

time: 5.24 s (started: 2023-10-27 16:03:15 +00:00)


In [ ]:
import ast
words = [ast.literal_eval(key) for key in corpus['keyword']]
#words = [key.split() for key in corpus['keyword']]
bm25 = BM25Okapi(words)
indexs = list(range(len(words)))

def bm25_searcher(question, corpus, bm25, indexes, top_k=1):
    #tokenized_query = question.split()
    tokenized_query = get_nouns_multipartite(question)
    #print(f'Question keys: {tokenized_query}')
    top_indexs = bm25.get_top_n(tokenized_query, indexes, n=top_k)

    top_docs = []
    top_summary = []
    for top_index in top_indexs:
      corpus_keys = corpus['keyword'][top_index]
      #print(f'Top index: {top_index}. Corpus keys: {corpus_keys}')
      top_docs.append(corpus['context'].iloc[top_index])
      top_summary.append(corpus['summary'].iloc[top_index])

    return top_docs, top_summary

time: 90.3 ms (started: 2023-10-27 16:47:42 +00:00)


In [ ]:
question = test_data.iloc[29]['question']
print(question)

import ast
words = [ast.literal_eval(key) for key in corpus['keyword']]
bm25 = BM25Okapi(words)
indexs = list(range(len(words)))

top_docs, _ = bm25_searcher(question, corpus, bm25, indexs, top_k=15)
top_docs

Thủ thuật Cryotheraphy (Liệu pháp lạnh) được sử dụng để loại bỏ nốt sùi mào gà bằng cách gì?.


['sùi mào gà ở nữ - điều trị bằng phẫu thuật. trong trường hợp điều trị bằng thuốc không mang lại hiệu quả, chị em sẽ được tư vấn phương pháp phẫu thuật để loại bỏ nốt sùi mào gà. hiện nay, tại',
 'sùi mào gà - triệu chứng bệnh sùi mào gà ở cả nam và nữ. theo các chuyên gia nam học, hệ thống',
 'sùi mào gà ở nữ - giai đoạn phát triển bệnh sùi mào gà ở nữ. bác sĩ nguyễn đức hinh cho biết, dựa theo quá trình phát triển và lây lan của các nốt mụn cóc mà bệnh sùi mào gà ở nữ được chia thành 5 giai đoạn như sau:',
 'sùi mào gà - cách phòng ngừa sùi mào gà. nếu đang trong độ tuổi có sinh hoạt tình dục, bạn có thể thực hiện các biện pháp sau đây để bảo vệ bản thân và người bạn đời khỏi nguy cơ bị nhiễm hoặc lây lan hpv và các bệnh lây truyền qua đường tình dục khác: ',
 'sùi mào gà ở nữ - nguyên nhân mắc sùi mào gà ở nữ giới?. bệnh sùi mào gà ở nữ giới là hậu quả của việc nhiễm virus hpv – một loại nhiễm trùng lây truyền qua đường tình dục (sti). theo',
 'sùi mào gà ở nữ - các biện pháp phòng

time: 3.01 s (started: 2023-10-27 17:36:16 +00:00)


#Get context for a question

In [ ]:
import ast
words = [ast.literal_eval(key) for key in corpus['keyword']]
bm25 = BM25Okapi(words)
indexs = list(range(len(words)))

retrieved_context = []
for index, row in test_data.iterrows():
  quest = row['question']
  top_docs, _ = bm25_searcher(quest, corpus, bm25, indexs, top_k=15)
  retrieved_context.append(" \\\ ".join(top_docs))
  print(f'finised quest : {quest}')

finised quest : Hương đang mang thai và lo lắng mình có thể gặp phải rau tiền đạo. Hương có thể kiểm tra phát hiện bệnh này từ tuần thứ mấy của thai kỳ?
finised quest : Hương đang mang thai tuần thứ 5 và lo lắng mình có thể gặp phải rau tiền đạo. Hương có thể kiểm tra phát hiện bệnh này sau bao nhiêu tuần nữa?
finised quest : Có bao nhiêu loại rau tiền đạo biết rằng trong bóng đá thường mỗi đội có tối đa 3 tiền đạo trên sân?
finised quest : Bệnh nhân Dũng được chuẩn đoán bị viêm gan kéo dài khoảng 4 tháng. Hỏi bệnh nhân Dũng có phải bị viêm gan mãn tính hay không?
finised quest : Một bệnh nhân bị đau tinh hoàn. Sau khi được hỏi thì tinh hoàn bên phải bệnh nhân hoàn toàn bình thường, bên còn lại có hiện tượng đau. Nguyên nhân chủ yếu của hiện tượng này là do:
finised quest : An bị viêm cột sống, còn Nam bị tiêu chảy. Hỏi bệnh của Nam thuộc nhóm bệnh lý nào?
finised quest : Việc sử dụng thuốc kháng sinh một cách tự tiện, quá liều lượng, không theo đơn có thể gây ra vấn đề nào?
finised qu

In [ ]:
test_data['retrieved_context'] = retrieved_context
test_data

In [ ]:
test_data.to_csv("updated_test_data.csv")

time: 52.5 ms (started: 2023-10-26 11:43:06 +00:00)


#Test

In [ ]:
import re
import string

def remove_url(text):
    return re.sub(r"http\S+", "", text)

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

codes = ["–", "&"]
def remove_special_token(text):
    for code in codes:
        text = text.replace(code, " ")
    return text

def remove_punctation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text

def preprocess_text(text, remove_punc=True, is_lower=True):
    if remove_punc:
        text = " ".join(remove_punctation(remove_special_token(remove_html_tags(remove_url(text)))).split())
    else:
        text = " ".join(remove_special_token(remove_html_tags(remove_url(text))).split())
    if is_lower:
        return text.lower()
    else:
        return text

time: 1.32 ms (started: 2023-10-26 11:45:11 +00:00)


In [ ]:
!pip install -q transformers sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.6 MB/s eta 0:00:00
time: 22.2 s (started: 2023-10-26 11:45:14 +00:00)


In [ ]:
from sentence_transformers import SentenceTransformer
#model = SentenceTransformer('keepitreal/vietnamese-sbert')
model = SentenceTransformer('all-MiniLM-L6-v2')

time: 3.87 s (started: 2023-10-26 12:05:49 +00:00)


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

model_name = 'xlm-roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

time: 6.38 s (started: 2023-10-26 12:05:55 +00:00)


In [ ]:
def embed_passage(passages, device='cpu'):
    # Tokenize sentences
    encoded_input = tokenizer(passages, padding=True, truncation=True, return_tensors='pt')
    #model = model.to(device)
    #encoded_input.to(device)
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    passage_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    return passage_embeddings

time: 993 µs (started: 2023-10-26 12:06:06 +00:00)


In [ ]:
question = test_data.iloc[5]['question']
print(question)

import ast
words = [ast.literal_eval(key) for key in corpus['keyword']]
bm25 = BM25Okapi(words)
indexs = list(range(len(words)))

top_docs, top_summary = bm25_searcher(question, corpus, bm25, indexs, top_k=15)
top_summary

An bị viêm cột sống, còn Nam bị tiêu chảy. Hỏi bệnh của Nam thuộc nhóm bệnh lý nào?
Question keys: ['nam', 'diseases', 'diarrhea', 'group', 'spondylitis']
Top index: 6414. Corpus keys: ['lysosomal storage disorders', 'function', 'body', 'enzymes', 'loss', 'cell structure', 'group', 'genes', 'diseases', 'absence']
Top index: 8734. Corpus keys: ['eye', 'uveitis', 'inflammatory diseases', 'group', 'swelling', 'retina', 'eye tissues', 'optic nerve', 'orbital pain', 'diseases']
Top index: 7400. Corpus keys: ['hematuria', 'bloody urine', 'symptom', 'urethral diseases', 'blood', 'bladder', 'group', 'kidney', 'clinical sign', 'diseases']
Top index: 7366. Corpus keys: ['blood', 'waste', 'urine', 'body', 'ability', 'diseases', 'kidneys', 'group', 'risk', 'long-term damage']
Top index: 6333. Corpus keys: ['diseases', 'mild', 'severe', 'levels', 'complex', 'autoimmune arthritis', 'group', 'stages', 'musculoskeletal system', 'dangerous complications']
Top index: 8026. Corpus keys: ['group', 'chroni

['Lysosomal storage disorders are a group of diseases related to about 40 disorders of genes encoding enzymes. Absence or loss of function of an enzyme will cause accumulation of intermediate metabolic products, leading to destruction of cell structure and function. Large number of cells die, causing lost organ function in the body.',
 "Uveitis includes a group of inflammatory diseases that cause swelling and destruction of eye tissues. These diseases can spread to other parts of the eye, including the retina and optic nerve - causing orbital pain and bruising if left untreated for more than ten days 'as soon as possible'",
 'Hematuria is a symptom of most diseases related to organs in the urinary system. Kidney and urethral diseases are the group of diseases that definitely cause bloody urine. In some other diseases such as bladder, blood in urine is clinical sign of bladder bleeding.',
 "Glomerulonephritis is a group of diseases that damage the body's ability to filter blood, waste a

time: 8.4 s (started: 2023-10-26 12:25:10 +00:00)


In [ ]:
process_paragraphs = [preprocess_text(doc) for doc in top_summary]
process_paragraphs

['lysosomal storage disorders are a group of diseases related to about 40 disorders of genes encoding enzymes absence or loss of function of an enzyme will cause accumulation of intermediate metabolic products leading to destruction of cell structure and function large number of cells die causing lost organ function in the body',
 'uveitis includes a group of inflammatory diseases that cause swelling and destruction of eye tissues these diseases can spread to other parts of the eye including the retina and optic nerve causing orbital pain and bruising if left untreated for more than ten days as soon as possible',
 'hematuria is a symptom of most diseases related to organs in the urinary system kidney and urethral diseases are the group of diseases that definitely cause bloody urine in some other diseases such as bladder blood in urine is clinical sign of bladder bleeding',
 'glomerulonephritis is a group of diseases that damage the body s ability to filter blood waste and urine patient

time: 6.02 ms (started: 2023-10-26 12:25:20 +00:00)


In [ ]:
passage_embeddings = embed_passage(process_paragraphs)

time: 7.24 s (started: 2023-10-26 12:25:24 +00:00)


In [ ]:
from sentence_transformers import util

time: 681 µs (started: 2023-10-26 11:51:16 +00:00)


In [ ]:
question = test_data.iloc[5]['question']
question_embedding = embed_passage([preprocess_text(ques_processing(question))])

time: 4.83 s (started: 2023-10-26 12:25:33 +00:00)


In [ ]:
top_k = 5
cos_scores = util.cos_sim(question_embedding, passage_embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k)

time: 3.23 ms (started: 2023-10-26 12:25:38 +00:00)


In [ ]:
print(question)
for idx in top_results.indices.tolist():
  print(process_paragraphs[idx])

An bị viêm cột sống, còn Nam bị tiêu chảy. Hỏi bệnh của Nam thuộc nhóm bệnh lý nào?
placenta previa can form anywhere in the uterus depending on the location if embryo implants in lower part of utorus it will not move upward during pregnancy there is a dangerous condition called praevia which is caused by abnormal conditions in placenda in 35 years and older
1 5 of cases of pregnant mothers with placenta previa do not have an old surgical wound on their upper body the uterine body is at risk of progressing to rcrl according to the number of surgeries performed on the body
blood vessels proliferate in the lower part of the uterus penetrating the bladder elective cesarean section is needed at full term to remove fetus from placenta previa which is considered the most severe clinical form of a c section
placenta previa clinging to the edge is a warning sign of the possibility of selling if the cervix is open it can be felt by the edges of your vegetables causing it to stick to your uterus

In [ ]:
import ast
words = [ast.literal_eval(key) for key in corpus['keyword']]
bm25 = BM25Okapi(words)
indexs = list(range(len(words)))

def get_top_context(question, corpus, top_k = 5):
  top_context = []
  top_docs, top_summary = bm25_searcher(question, corpus, bm25, indexs, top_k=15)
  process_paragraphs = [preprocess_text(doc) for doc in top_summary]
  passage_embeddings = embed_passage(process_paragraphs)
  question_embedding = embed_passage([preprocess_text(ques_processing(question))])
  cos_scores = util.cos_sim(question_embedding, passage_embeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  for idx in top_results.indices.tolist():
    top_context.append(top_docs[idx])
  return ' /// '.join(top_context)

time: 1.03 s (started: 2023-10-26 12:28:18 +00:00)


In [ ]:
question = test_data.iloc[0]['question']
print(question)
get_top_context(question, corpus, top_k = 5)

Hương đang mang thai và lo lắng mình có thể gặp phải rau tiền đạo. Hương có thể kiểm tra phát hiện bệnh này từ tuần thứ mấy của thai kỳ?


'rau bám mép có nguy hiểm không? nguyên nhân, dấu hiệu, phòng ngừa - nguyên nhân dẫn đến rau bám mép. hiện tại, nguyên nhân dẫn đến rau tiền đạo bám mép nói riêng và rau tiền đạo nói chung vẫn chưa được xác định rõ ràng.bác sĩ hữu côngcho biết, rau thai có thể hình thành ở vị trí nào trong lòng tử cung theo vị trí mà phôi thai làm tổ. điều đó có nghĩa là, nếu phôi thai làm tổ ở đoạn dưới của tử cung thì nhau thai sẽ phát triển ở ngay vị trí này và không xê dịch lên phía trên trong suốt thai kỳ. kết quả là dẫn đến tình trạng rau tiền đạo. bên cạnh đó, một số yếu tố làm tăng nguy cơ bị rau tiền đạo gồm: tham khảo: những tình trạng bất thường ở bánh nhau nguy hiểm phụ nữ mang thai từ 35 tuổi trở lên; phụ nữ trải qua sinh nở nhiều lần; tiền sử từng sảy thai hoặc nạo phá thai nhiều lần; tiền sử viêm nhiễm cổ tử cung; tiền sử bị rau tiền đạo ở những lần mang thai trước; mẹ bầu mang song thai hoặc đa thai, có bánh nhau lớn sẽ tăng nguy cơ mắc bệnh; hình dạng tử cung bất thường; lối sống và si

time: 17.8 s (started: 2023-10-26 12:30:19 +00:00)
